In [1]:
from empath import Empath
lexicon = Empath()
import pandas as pd
import csv
import numpy as np


In [2]:

raw_news1 = pd.read_csv("articles1.csv")
raw_news2 = pd.read_csv("articles2.csv")
raw_news3 = pd.read_csv("articles3.csv")

print(raw_news1.shape[0])

50000


In [3]:
def change_date_expression(date):
    if date[4] == "-":
        return date
    
    break_point = []
    for i in range(len(date)):
        if date[i] == '/':
            break_point.append(i)
    
    if len(break_point) != 0:
        out = date[0:break_point[0]] + '-' + date[break_point[0]+1:break_point[1]]+'-'+date[break_point[1]+1:]
        return out
    else:
        return date

In [4]:
def count_day(dataset,dict_count):
    for i in range(dataset.shape[0]):
        date = dataset[i][5];
        if dict_count.__contains__(date):
            dict_count[date] += 1
        else:
            dict_count[date] = 1

In [5]:
def relevance_caculation(dic):
    relavences = ["social_media","computer","business","programming","hearing","urban","shopping","science","work","valuable","fashion","technology","competing","economics","office"]
    negetive_attitude = ['hate','aggression','horror','suffering','ridicule','irritability','deception','disappointment','negative_emotion','nervousness']
    positive_attitude = ['cheerfulness','optimism','celebration','trust','positive_emotion']
    
    # correlation c:
    c = 0
    for relavence in relavences:
        c += dic[relavence]
    c = float(c) / float(len(relavences))
    
    # negetive attitude:
    neg_att = 0
    for neg in negetive_attitude:
        neg_att += dic[neg]
    neg_att = float(neg_att) / float(len(negetive_attitude))
    
    # positive attitude:
    pos_att = 0
    for pos in positive_attitude:
        pos_att += dic[pos]
    pos_att = float(neg_att) / float(len(positive_attitude))
    
    return float(c*100), float(c*neg_att*1000), float(c*pos_att*1000)

In [6]:
def pre_processing_dataset(dataset):
    # first we need to drop and add some columns
    # the colums we need to drop is "id", "title", "publication", "author", "year", "month", "url", "Unnamed: 0"
    dataset = dataset.drop(columns = ["id", "title", "publication", "author", "year", "month", "url", "Unnamed: 0"])
    dataset["correlation"] = 0
    dataset["positive_relavent"] = 0
    dataset["negetive_relavent"] = 0
    
    dataset["correlation"] = dataset.correlation.astype(float)
    dataset["positive_relavent"] = dataset.positive_relavent.astype(float)
    dataset["negetive_relavent"] = dataset.negetive_relavent.astype(float)
    return dataset

In [7]:
def processing_analysis_with_only_three_value(dataset):
    # the colums we need to add is sentiment analysis result 
    # plus three special colums, which are correlation c, positive attitude p * c, negetive attitude n *c.
    # second we need to change the all the date style into 2017-1-1
    # all this need to do in a loop:
    for i in range(dataset.shape[0]):
        # change date:
        if pd.isna(dataset["date"][i]) == True:
            print(dataset["date"][i])
        else:
            dataset["date"][i] = change_date_expression(dataset["date"][i])
        
        # analysis the contont:
        ana_dict = lexicon.analyze(dataset["content"][i], normalize = True)
        #for key, value in ana_dict.items():
            #dataset[key][i] = value
        # puls three special values.
        if ana_dict is None:
            dataset["correlation"][i] = 0
            dataset["positive_relavent"][i] = 0
            dataset["negetive_relavent"][i] = 0
        else:
            c, neg, pos = relevance_caculation(ana_dict)
            dataset["correlation"][i] = c
            dataset["positive_relavent"][i] = neg
            dataset["negetive_relavent"][i] = pos
        
        if i % 1000 == 0:
            print(i)
            print(dataset["date"][i])
        
    return dataset 

In [8]:
def out_put_result(dataframe, n):
    file_name = str(n)+"_part_anaylsis.csv"
    
    dataframe1 = pre_processing_dataset(dataframe)
    dataframe1_with_analysis = processing_analysis(dataframe1)
    dataframe1_with_analysis = dataframe1_with_analysis.drop(columns = ["content"])
    
    dataframe1_with_analysis.to_csv(file_name, encoding='utf-8')
    
    return dataframe1_with_analysis

In [12]:
# ["social_media","computer","business","programming","hearing","urban","shopping","science","work",
#      "valuable","fashion","technology","competing","economics","office"]
# ['hate','aggression','horror','suffering','ridicule','irritability','deception',
#      'disappointment','negative_emotion','nervousness']
# ['cheerfulness','optimism','celebration','trust','positive_emotion']
# now what i should do is keep all this values
# so need dataframe with columns: "date", ""social_media","computer","business","programming","hearing",
#       "urban","shopping","science","work",
#      "valuable","fashion","technology","competing","economics","office", 'hate','aggression',
#       'horror','suffering','ridicule','irritability','deception',
#      'disappointment','negative_emotion','nervousness', 'cheerfulness','optimism','celebration','trust',
#      'positive_emotion',

In [22]:
def processing_analysis_with_all_variables(dataset):
    # pre-processing the dataset for droping useless colums
    dataset = dataset.drop(columns = ["id", "title", "publication", "author", "year", "month", "url", "Unnamed: 0"])
    # we need to have a list hold all the parameters
    check_list = ["social_media","computer","business","programming","hearing","urban","shopping","science","work","valuable","fashion","technology","competing","economics","office",'cheerfulness','optimism','celebration','trust','positive_emotion','hate','aggression','horror','suffering','ridicule','irritability','deception', 'disappointment','negative_emotion','nervousness']    
    # inital matrix that have all the columens:
    data_list = ["date-month-day"]
    for parameter in check_list:
        data_list.append(np.float16(0))
    
    for i in range(3):
        data_list.append(np.float16(0))
        
    data_matrix = np.array([data_list])
    
    print(data_matrix)
    
    for i in range(dataset.shape[0]):
        # change date:
        if pd.isna(dataset["date"][i]) == True:
            print(dataset["date"][i])
        else:
            dataset["date"][i] = change_date_expression(dataset["date"][i])
        
        # analysis the contont:
        ana_dict = lexicon.analyze(dataset["content"][i], normalize = True)
        #for key, value in ana_dict.items():
            #dataset[key][i] = value
        # puls three special values.
        
        if ana_dict is not None:
            one_line_list = []
            one_line_list.append(dataset["date"][i])
            for parameter in check_list:
                one_line_list.append(ana_dict[parameter])
            relev, pos, neg = relevance_caculation(ana_dict)
            one_line_list.append(relev)
            one_line_list.append(pos)
            one_line_list.append(neg)
            
            data_matrix = np.insert(data_matrix, 0, [one_line_list], 0)
        
        if i % 1000 == 0:
            print(i)
            print(dataset["date"][i])
    
    dataframe = pd.DataFrame(data_matrix, columns=['date', "social_media","computer","business","programming","hearing","urban","shopping","science","work","valuable","fashion","technology","competing","economics","office",'cheerfulness','optimism','celebration','trust','positive_emotion','hate','aggression','horror','suffering','ridicule','irritability','deception', 'disappointment','negative_emotion','nervousness' ,'correlation', 'positive_relavent','negetive_relavent'])

    return dataframe



In [23]:
def get_final_result_into_csv(dataframe, n):
    file_name = str(n)+"_part_anaylsis_with_all_parameters.csv"
    
    dataframe1_with_analysis = processing_analysis_with_all_variables(dataframe)
    
    dataframe1_with_analysis.to_csv(file_name, encoding='utf-8')
    
    return dataframe1_with_analysis

In [20]:
news2 = get_final_result_into_csv(raw_news2,2)

news2.head(10)

[['date-month-day' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0'
  '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0'
  '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0']]
0
2017-01-11
1000
2017-03-09
2000
2016-01-30
3000
2016-05-20
4000
2016-06-26
5000
2016-08-25
6000
2016-10-24
7000
2016-12-29
nan
nan
8000
2016-03-26
9000
2016-06-25
10000
2016-09-13
11000
2016-11-26
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
na

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


,date,social_media,computer,business,programming,hearing,urban,shopping,science,work,...,suffering,ridicule,irritability,deception,disappointment,negative_emotion,nervousness,correlation,positive_relavent,negetive_relavent
0,2016-07-15,0.001427212178,0.000951474785,0.003805899143,0.000951474785,0.001427212178,0.000951474785,0.001902949571,0.000475737392,0.002378686964,...,0.002854424357,0.0,0.001427212178,0.002378686964,0.0,0.006660323501,0.001902949571,0.152235965746,0.002534851951,0.000506970390
1,2016-12-26,0.0,0.000981354268,0.001962708537,0.001962708537,0.001962708537,0.000981354268,0.000981354268,0.0,0.001962708537,...,0.0,0.0,0.000981354268,0.000981354268,0.003925417075,0.008832188420,0.0,0.143931959437,0.003107461342,0.000621492268
2,2016-09-16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.069444444444,0.020833333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.601851851851,0.0,0.0
3,2016-08-28,0.0,0.001785714285,0.005357142857,0.0,0.001785714285,0.0,0.005357142857,0.0,0.001785714285,...,0.0,0.0,0.0,0.0,0.0,0.005357142857,0.003571428571,0.166666666666,0.002083333333,0.000416666666
4,2016-08-12,0.0015625,0.0,0.0046875,0.0015625,0.0015625,0.0046875,0.0,0.0,0.0140625,...,0.0046875,0.0015625,0.0,0.0,0.0,0.0015625,0.0,0.229166666666,0.004296875000,0.000859375000
5,2016-07-03,0.006230529595,0.0,0.001557632398,0.0,0.001557632398,0.003115264797,0.0,0.0,0.001557632398,...,0.001557632398,0.0,0.0,0.0,0.0,0.001557632398,0.001557632398,0.197300103842,0.000921963102,0.000184392620
6,2016-10-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005235602094,0.0,0.007853403141,0.002617801047,0.0,0.007853403141,0.0,0.104712041884,0.004659959979,0.000931991995
7,2016-07-02,0.0,0.0,0.010204081632,0.0,0.0,0.0,0.0,0.0,0.013605442176,...,0.001700680272,0.0,0.0,0.001700680272,0.0,0.003401360544,0.0,0.215419501133,0.002930877566,0.000586175513
8,2016-11-27,0.002763194252,0.0,0.003039513677,0.0,0.008289582757,0.000552638850,0.001105277701,0.000552638850,0.001934235976,...,0.003315833103,0.000828958275,0.000276319425,0.000552638850,0.000276319425,0.009118541033,0.002210555402,0.149212489638,0.002927351965,0.000585470393
9,2016-11-11,0.00125,0.00125,0.0125,0.00125,0.0,0.0,0.0025,0.0,0.005,...,0.00125,0.0025,0.0,0.00875,0.0,0.005,0.0,0.350000000000,0.007875000000,0.001575000000


In [21]:
news3 = get_final_result_into_csv(raw_news3,3)

news3.head(10)

[['date-month-day' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0'
  '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0'
  '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0']]
0
2016-07-13
nan
nan
nan
nan
nan
nan
nan
nan
nan
1000
2016-12-11
nan
nan
nan
2000
2016-09-02
nan
nan
3000
2016-11-28
nan
4000
2017-01-09
5000
2017-02-17
6000
2017-04-01
7000
2017-05-15
8000
2017-06-27
9000
2016-02-10
10000
2016-03-31
11000
2016-05-23
12000
2016-07-12
13000
2016-08-26
14000
2016-10-09
15000
2016-11-22
16000
2017-01-10
17000
2017-02-16
18000
2017-03-28
19000
2017-05-09
20000
2017-06-19
21000
2016-01-31
22000
2016-03-21
23000
2016-05-05
24000
2016-09-07
25000
2016-11-04
26000
2016-12-09
27000
2017-2-12
28000
2017-4-24
29000
2016-3-28
30000
2016-7-22
31000
2016-11-14
32000
2017-01-25
33000
2017-03-14
34000
2017-05-01
35000
2017-06-18
36000
2016-02-05
37000
2016-04-01
38000
2016-05-25
39000
2016-07-13
40000
2016-08-28
41000
2016-10-14
42000
2016-12-01


,date,social_media,computer,business,programming,hearing,urban,shopping,science,work,...,suffering,ridicule,irritability,deception,disappointment,negative_emotion,nervousness,correlation,positive_relavent,negetive_relavent
0,2016-12-29,0.0,0.0,0.002528445006,0.002528445006,0.001264222503,0.0,0.003792667509,0.011378002528,0.002528445006,...,0.0,0.0,0.0,0.0,0.0,0.003792667509,0.0,0.244416350611,0.001544983252,0.000308996650
1,2016-12-30,0.001461988304,0.0,0.010233918128,0.0,0.0,0.005847953216,0.005847953216,0.0,0.001461988304,...,0.001461988304,0.0,0.0,0.004385964912,0.0,0.001461988304,0.0,0.253411306042,0.001852421827,0.000370484365
2,2016-12-30,0.0,0.009149130832,0.019213174748,0.010978956999,0.0,0.003659652333,0.0,0.009149130832,0.022872827081,...,0.000914913083,0.0,0.0,0.0,0.000914913083,0.002744739249,0.0,0.969807868252,0.005323739441,0.001064747888
3,2016-12-30,0.0,0.001742160278,0.008710801393,0.0,0.001742160278,0.0,0.0,0.0,0.008710801393,...,0.0,0.0,0.0,0.001742160278,0.001742160278,0.001742160278,0.0,0.185830429732,0.000971239179,0.000194247835
4,2016-12-30,0.002544529262,0.0,0.003816793893,0.0,0.003816793893,0.002544529262,0.0,0.0,0.002544529262,...,0.001272264631,0.0,0.0,0.001272264631,0.001272264631,0.001272264631,0.0,0.161153519932,0.001025149617,0.000205029923
5,2016-12-30,0.005175983436,0.002070393374,0.003105590062,0.001035196687,0.002070393374,0.001035196687,0.0,0.001035196687,0.002070393374,...,0.002070393374,0.0,0.0,0.0,0.0,0.001035196687,0.001035196687,0.151828847481,0.000943036319,0.000188607263
6,2016-12-30,0.001203369434,0.0,0.020457280385,0.002406738868,0.0,0.002406738868,0.004813477737,0.0,0.002406738868,...,0.0,0.0,0.0,0.0,0.0,0.003610108303,0.0,0.489370236662,0.002355572739,0.000471114547
7,2016-12-30,0.001018329938,0.0,0.012219959266,0.0,0.001018329938,0.001018329938,0.0,0.0,0.008146639511,...,0.0,0.0,0.0,0.001018329938,0.001018329938,0.003054989816,0.0,0.251188051595,0.004348469324,0.000869693864
8,2016-12-30,0.001821493624,0.005464480874,0.005464480874,0.003642987249,0.001821493624,0.0,0.0,0.001821493624,0.003642987249,...,0.003642987249,0.0,0.003642987249,0.003642987249,0.003642987249,0.0,0.001821493624,0.291438979963,0.007962813660,0.001592562732
9,2016-12-30,0.0,0.001291989664,0.010335917312,0.001291989664,0.011627906976,0.0,0.001291989664,0.001291989664,0.007751937984,...,0.005167958656,0.0,0.0,0.001291989664,0.0,0.002583979328,0.006459948320,0.327304048234,0.010148962735,0.002029792547
